In [ ]:
"""
Most recent version of m/m/2 queue simulation
*takes several seconds to compile (output is at bottom of cell)

"""
import cvxpy
import numpy as np
import random as rm
import random
from scipy.stats import poisson
from scipy.stats import expon
from re import X
class MailroomSim:
    #m/m/2 queue with exponential distribution (mean 1) for arrival and service times
    def __init__(self):
      """
      Initialize class instances below
      num_students: number of students waiting in line
      clock: the time in simulation in the unit of minutes. The clock acts as a real clock, accumulates +1 in every minute
      employee1_available_time / employee2_available_time: the amount of time that the staff needs to finish the current service
            Example: if employee1_available_time is 840 and current time(clock) is 800, then the employee will be available at 840, 40 minutes after
            Currently have two such instances, meaning two staffs
      arrival_times_list: 
      departure_times_list:
      queue_length_list
      """
      self.num_students = 0
      self.clock = 0.0

     
      self.employee1_available_time = 0.0
      self.employee2_available_time = 0.0

      self.arrival_times_list = []
      self.departure_times_list = []
      self.queue_length_list = []
      self.service_time_list = []


    def increment_time(self): 
      """
      Creates output of all arrival and departure times within 8am-10pm Monday-Friday business hours
      Generates arrival times in an expotential distrbution 
      """
      count_time = 0.0

      random_exp_list = poisson.rvs(mu=1, size=10000)
      while count_time <= 840:
        count_time += np.random.choice(random_exp_list)
        if count_time <= 840:
          self.arrival_times_list.append(count_time)
      
      print_every_min = 0
      while self.clock < 840:
        arrivals_so_far = 0
        departures_so_far = 0

        for x in self.arrival_times_list: #exits loop if there is a event that occurs after the present self.clock
          if x <= self.clock:
            arrivals_so_far += 1
          else:
            break

        for y in self.departure_times_list:
          if y <= self.clock:
            departures_so_far += 1
          else:
            break

        self.num_students = arrivals_so_far - departures_so_far


        if self.num_students == 1 and self.employee1_available_time <= self.clock and self.employee2_available_time <= self.clock: #student has not already been served
            requests_fulfilled = self.process_request(False)
            self.num_students -= 1
        elif self.num_students >= 2:
            if self.num_students == 2 and (self.employee1_available_time <= self.clock and self.employee2_available_time > self.clock) or (self.employee2_available_time <= self.clock and self.employee1_available_time > self.clock):
              requests_fulfilled = self.process_request(False) #effectively just one student being served, as one has already been accounted for
              self.num_students -= 1
            elif self.employee1_available_time <= self.clock and self.employee2_available_time <= self.clock:
              requests_fulfilled = self.process_request(True)
              self.num_students -= 2
            elif self.num_students > 2:
              requests_fulfilled = self.process_request(True)  
              self.num_students -= 2     

        temp_calculation = self.clock - print_every_min
        if (temp_calculation <= 1 and temp_calculation >= 0) or (temp_calculation >= -1 and temp_calculation <= 0 and self.num_students >= 0):
          self.queue_length_list.append(self.num_students)
          print_every_min += 10

        self.clock += 0.01
      time_label = ""
      hours = ""
      minutes = ""
      AM_or_PM = ""

      print("Average Number of People in System:")
      print(sum(self.queue_length_list)/len(self.queue_length_list))
      print("Average Service Time:")
      print((sum(self.service_time_list))/len(self.service_time_list))
      print("Arrivals/Hour")
      print(len(self.arrival_times_list)/(14))
      print("Customers Served/Hour: "+ str(len(self.departure_times_list)/14))


      #formats into AM or PM
      for x in self.arrival_times_list:
        if x/60 + 8 > 12: #1 pm and after
          hours = str(int(x/60) + 8 - 12)
          AM_or_PM = "PM"
        else: #12 or before (but 12 is modified again later)
          hours = str(int(x/60) + 8)
          AM_or_PM = "AM"

        if int(x/60) + 8 == 12: #assignments for 12pm
          hours = str(12)
          AM_or_PM = "PM"

        if int(x%60 < 10): #minutes formatting
          minutes = "0" + str(int(x%60))
        else:
          minutes = str(int(x%60))
        printed_time = hours + ":" + minutes + " " + AM_or_PM

      for x in self.departure_times_list:
        if x/60 + 8 > 12:
          hours = str(int(x/60) + 8 - 12)
          AM_or_PM = "PM"
        else:
          hours = str(int(x/60) + 8)
          AM_or_PM = "AM"

        if int(x/60) + 8 == 12:
          hours = str(12)
          AM_or_PM = "PM"

        if int(x%60 < 10):
          minutes = "0" + str(int(x%60))
        else:
          minutes = str(int(x%60))
        printed_time = hours + ":" + minutes + " " + AM_or_PM

    #employee functions to process requests
    def process_request(self, multiple_students): #function is only called if at least one person should be served
      employees_called = 0
      #using e1 e2 as abbreviation for employees
      if self.employee1_available_time <= self.clock: #if function called, always either e1 is available for service or isn't
        service_time = self.service_time()
        self.service_time_list.append(service_time)
        self.employee1_available_time = self.clock + service_time
        self.departure_times_list.append(self.employee1_available_time)
        employees_called = 1

      if ((multiple_students == False and employees_called == 0) or multiple_students == True) and self.employee2_available_time <= self.clock: #not used if e2 busy or 1 student and e1 already called
        service_time = self.service_time()
        self.service_time_list.append(service_time)
        self.employee2_available_time = self.clock + service_time
        self.departure_times_list.append(self.employee2_available_time)
        employees_called += 1

      return employees_called


    #how long something takes to process depending on mail or other request (values can be modified based on observed real data)
    def service_time(self):
      return np.random.choice(poisson.rvs(mu=1, size=10000))

s = MailroomSim()
s.increment_time()

Average Number of People in System:
1.4588235294117646
Average Service Time:
1.0392609699769053
Arrivals/Hour
61.92857142857143
Customers Served/Hour: 61.857142857142854


In [ ]:
"""
Alternate queue simulation unrelated to most recent version (used initially, but unlike the most recent version, does not implement exponential distributions of arrival and service times)

"""

def mailroom_simulation(frequency, time):
  # frequency: the probability that a student will come to RPCC to retrieve package; out of 100
  student_inline = 0
  total_arrival = 0

  # TIME_MIN: represents the actual time in minutes counting from 0 to 1440
  global TIME_MIN
  TIME_MIN = 0

  staff1_available = True
  staff2_available = True

  total_student_inRPCC = 0
  total_students_log = 0
  total_student_served = 0
  
  count_time = 0
  random_exp_list = np.random.exponential(1,500)
  while count_time < 840:
    count_time += np.random.choice(random.exp_list)


  # -1 is the default time for waiting_time
  waiting_time1 = -1
  waiting_time2 = -1
  waiting_time3 = -1

  # 1440 because there are 1440 minutes in a day, but we definitely can make it more resonable to be the actual open time.=
  while TIME_MIN <= time:
    TIME_MIN +=1

    #the time students come to RPCC is totally random
    # currently sets this to be about 35%
    if random.random()*100 < frequency:
      total_student_inRPCC +=1
      total_students_log +=1
      student_inline +=1

    # check if the student can just be served by an available staff
    if staff1_available and student_inline > 0:
        waiting_time1 = 0
        staff1_available = False
        student_inline -=1
        waiting_time1 = TIME_MIN + int(random.random()*5+3)

    elif staff2_available and student_inline > 0:
        waiting_time2 = 0
        staff2_available = False
        student_inline -=1
        waiting_time2 = TIME_MIN + int(random.random()*5+3)

  
    # if not, the student has to wait in line and check if the staffs are available in the next minute

    # after the student gets served, in which the waiting time is passed, student leaves. Staff back to available status
    if TIME_MIN == waiting_time1:
    
      staff1_available = True
      total_student_served += 1
      total_student_inRPCC -=1
  
    if TIME_MIN == waiting_time2:

      staff2_available = True
      total_student_served +=1 
      total_student_inRPCC -=1

    if TIME_MIN == waiting_time3:
      staff3_available = True

      total_student_inRPCC -=1
  
  
    #Print Statements
    print('When time is '+str(TIME_MIN)+ ', there are a total of '+str(student_inline)+' number of students in line. The total number of students in RPCC is '+str(total_student_inRPCC)+'. \
      The total number of students is '+ str(total_students_log) + '. The total number of students served by the staff is '+ str(total_student_served))
    print('The time for staff1 to get the package is '+str(waiting_time1))
    print('The time for staff2 to get the package is '+str(waiting_time2))
    print('The time for staff3 to get the package is '+str(waiting_time3))

  
  mailroom_simulation(frequency=35, time=1440)
      
